In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install -y python-enchant

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko python-gobject python-gtk2
  python-wxgtk3.0
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
  python-enchant
0 upgraded, 11 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,354 kB of archives.
After this operation, 5,584 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/ma

In [3]:
# 제일 먼저 tesseract 설치한다
!sudo apt install tesseract-ocr
!pip install pytesseract 
!pip install opencv-python
!pip install imutils
!pip install numpy
!pip install pillow
!pip install nltk
!pip install tensorflow
!pip install PyEnchant
!pip install langdetect

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,920 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
# 필요한 모듈 
import os
import cv2
import pytesseract
from nltk.tokenize import WordPunctTokenizer
import re
import enchant
from google.colab.patches import cv2_imshow
import imutils
from langdetect import detect_langs
%matplotlib inline

### **OCR**

In [ ]:
class OCR1():
  def __init__(self, img):
    self.config = r'--oem 3 --psm 3 -c preserve_interword_spaces=1'
    if img.shape[1] < 500:
      self.image = cv2.cvtColor(imutils.resize(img, width=500), cv2.COLOR_BGR2GRAY)
    else:
      self.image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_imshow(self.image)
    self.result = ''
    self.filter_math = '[^A-Za-z0-9+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x()]'
    self.language = enchant.Dict("en_US")
    self.english_score = []
    self.hindi_score = []

  def image_pytesseract(self):
    if not self.text_filtering():
      print("\ntwice text_filtering")
     
      blur =  cv2.GaussianBlur(self.image, (5,5), 0)
      self.image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      cv2_imshow(self.image)
      self.text_filtering()

  def text_filtering(self):
    self.result = WordPunctTokenizer().tokenize(pytesseract.image_to_string(self.image, lang='eng', config=self.config))
    #print("Original Nltk tokenizer Before : ",self.result)
    self.result = [re.sub(self.filter_math, '', i).lower() for i in self.result if re.sub(self.filter_math, '', i) != '']
    #print("After Nltk tokenizer, filtering : ", self.result)
    
    if len(self.result) <= 3: 
      print("False\n")
      return False
    else:
      Ncount = 0
      Ccount = 0
      Tcount = 0
      Fcount = 0
      for i in self.result:
        if i.isdigit(): #숫자
          print("this is num: ",i)
          Ncount += 1
        else: #숫자 외 -> 연산자 or 영어 단어
          if i in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.()':
            print("this is math operations : ", i, end="\n")
            Ccount += 1
          else:
            print("this is character : ", i,self.language.check(i),end="\n")
            if self.language.check(i): Tcount += 1
            else: Fcount += 1
      print("\ncount: ", Ncount)
      print("Ccount: ",Ccount)
      print("Tcount: ",Tcount)
      print("Fcount: ",Fcount)
      
      if (Ccount + Ncount + Tcount < 6) or ((Ccount + Ncount < Tcount + Fcount) and (Tcount <= Fcount)): # false일 경우
        print("False")
        return False
      else:
        print("True")
        return True


In [ ]:
# gray + gaussian + threshold

class OCR2():
  def __init__(self, img):
    self.config = r'--oem 3 --psm 3 -c preserve_interword_spaces=1'
    if img.shape[1] < 500:
      self.img = cv2.cvtColor(imutils.resize(img, width=500), cv2.COLOR_BGR2GRAY)
    else:
      self.img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    self.blur =  cv2.GaussianBlur(self.img, (5,5), 0)
    self.image = cv2.threshold(self.blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    cv2_imshow(self.image)
    self.result = ''
    self.filter_math = '[^A-Za-z0-9+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x]'
    self.language = enchant.Dict("en_US")

  def image_pytesseract(self):
    if not self.text_filtering():
      print("\ntwice text_filtering")
      self.image = self.img
      cv2_imshow(self.image)
      self.text_filtering()

  def text_filtering(self):
    self.result = WordPunctTokenizer().tokenize(pytesseract.image_to_string(self.image, lang='eng', config=self.config))
    print("Original Nltk tokenizer Before : ",self.result)
    self.result = [re.sub(self.filter_math, '', i).lower() for i in self.result if re.sub(self.filter_math, '', i) != '']
    print("After Nltk tokenizer, filtering : ", self.result)
    
    if len(self.result) <= 3: 
      print("False\n")
      return False
    else:
      Ncount = 0
      Ccount = 0
      Tcount = 0
      Fcount = 0
      for i in self.result:
        if i.isdigit(): #숫자
          print("this is num: ",i)
          Ncount += 1
        else: #숫자 외 -> 연산자 or 영어 단어
          if i in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.':
            print("this is math operations : ", i, end="\n")
            Ccount += 1
          else:
            print("this is character : ", i,self.language.check(i),end="\n")
            if self.language.check(i): Tcount += 1
            else: Fcount += 1
      print("\ncount: ", Ncount)
      print("Ccount: ",Ccount)
      print("Tcount: ",Tcount)
      print("Fcount: ",Fcount)
      
      if (Ccount + Ncount + Tcount < 6) or ((Ccount + Ncount < Tcount + Fcount) and (Tcount <= Fcount)): # false일 경우
        print("False")
        return False
      else:
        print("True")
        return True


In [ ]:
# 조건문 추가하기

class OCR3():
  def __init__(self, img):
    self.config = r'--oem 3 --psm 3 -c preserve_interword_spaces=1'
    if img.shape[1] < 500:
      self.image = cv2.cvtColor(imutils.resize(img, width=400), cv2.COLOR_BGR2GRAY)
    else:
      self.image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_imshow(self.image)
    self.result = ''
    self.filter_math = '[^A-Za-z0-9+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x]'
    self.language = enchant.Dict("en_US")

  def image_pytesseract(self):
    if not self.text_filtering():
      print("\ntwice text_filtering")
     
      blur =  cv2.GaussianBlur(self.image, (5,5), 0)
      self.image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      cv2_imshow(self.image)
      self.text_filtering()

  def text_filtering(self):
    self.result = WordPunctTokenizer().tokenize(pytesseract.image_to_string(self.image, lang='eng', config=self.config))
    print("Original Nltk tokenizer Before : ",self.result)
    self.result = [re.sub(self.filter_math, '', i).lower() for i in self.result if re.sub(self.filter_math, '', i) != '']
    print("After Nltk tokenizer, filtering : ", self.result)
    
    if len(self.result) <= 3: 
      print("False\n")
      return False
    else:
      Ncount = 0
      Ccount = 0
      Tcount = 0
      Fcount = 0
      for i in self.result:
        if i.isdigit(): #숫자
          print("this is num: ",i)
          Ncount += 1
        else: #숫자 외 -> 연산자 or 영어 단어
          if i in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.()':
            print("this is math operations : ", i, end="\n")
            Ccount += 1
          else:
            temp1, temp2 = [idx for idx in range(len(i)) if i[idx].isdigit()], [idx for idx in range(len(i)) if i[idx] in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.()']
            print("this is character : ", i,self.language.check(i),end="\n")
            if temp1 or temp2:
              Ncount += len(temp1)
              Ccount += len(temp2)
            else:
              if self.language.check(i): Tcount += 1
              else: Fcount += 1
      print("\ncount: ", Ncount)
      print("Ccount: ",Ccount)
      print("Tcount: ",Tcount)
      print("Fcount: ",Fcount)
      
      if ((Ccount + Ncount < Tcount + Fcount) and (Tcount <= Fcount)) or ((Tcount + Fcount >= 15) and (abs(Tcount - Fcount) <= 3)): # false일 경우 | (Ccount + Ncount + Tcount < 6) or 
        print("False")
        return False
      else:
        print("True")
        return True


In [16]:
# 조건문 추가하기

class OCR4():
  def __init__(self, img):
    self.config = r'--oem 3 --psm 3 -l eng+hin -c preserve_interword_spaces=1'
    if img.shape[1] < 500:
      self.image = cv2.cvtColor(imutils.resize(img, width=400), cv2.COLOR_BGR2GRAY)
    else:
      self.image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_imshow(self.image)
    self.result = ''
    self.filter_math = '[^A-Za-z0-9+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x]'
    self.language = enchant.Dict("en_US")

  def image_pytesseract(self):
    if not self.text_filtering():
      print("\ntwice text_filtering")
     
      self.image =  cv2.GaussianBlur(self.image, (5,5), 0)
      #self.image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
      cv2_imshow(self.image)
      self.text_filtering()

  def text_filtering(self):
    text = pytesseract.image_to_string(self.image, config=self.config)#lang='eng+hin',
    try:
      print("detect lang: ", detect_langs(text))
      
    except: return
    self.result = WordPunctTokenizer().tokenize(text)
    print("Original Nltk tokenizer Before : ",self.result)
    self.result = [re.sub(self.filter_math, '', i).lower() for i in self.result if re.sub(self.filter_math, '', i) != '']
    print("After Nltk tokenizer, filtering : ", self.result)
    
    if len(self.result) <= 3: 
      print("False\n")
      return False
    else:
      Ncount = 0
      Ccount = 0
      Tcount = 0
      Fcount = 0
      for i in self.result:
        if i.isdigit(): #숫자
          print("this is num: ",i)
          Ncount += 1
        else: #숫자 외 -> 연산자 or 영어 단어
          if i in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.()':
            print("this is math operations : ", i, end="\n")
            Ccount += 1
          else:
            temp1, temp2 = [idx for idx in range(len(i)) if i[idx].isdigit()], [idx for idx in range(len(i)) if i[idx] in '+-/*÷=×±∓∘∙∩∪≅∀√%∄∃θπσ≠<>≤≥≡∼≈≢∝≪≫∈∋∉⊂⊃⊆⊇⋈∑∫∏∞x,.()']
            print("this is character : ", i,self.language.check(i),end="\n")
            if temp1 or temp2:
              Ncount += len(temp1)
              Ccount += len(temp2)
            else:
              if self.language.check(i): Tcount += 1
              else: Fcount += 1
      print("\ncount: ", Ncount)
      print("Ccount: ",Ccount)
      print("Tcount: ",Tcount)
      print("Fcount: ",Fcount)
      
      if ((Ccount + Ncount < Tcount + Fcount) and (Tcount <= Fcount)) or ((Tcount + Fcount >= 10) and (abs(Tcount - Fcount) < 3)): # false일 경우 | (Ccount + Ncount + Tcount < 6) or 
        print("False")
        return False
      else:
        print("True")
        return True


### **real data**

In [ ]:
name = ['general_Capture','hindi','general_photo','image','handwritten']
path_dir = '/content/drive/My Drive/OCR/TEST_OCR1/real_Test/'
file_list = os.listdir(path_dir)
file_list = [(i, [path_dir + i + '/' + j for j in os.listdir(path_dir+i)]) for i in file_list]
file_list

OCR3가 가장 정확함

아래는 OCR3 테스트 결과 의미


텍스트이미지 부분
<br>
t24 -> 화질?<br>
t22 -> ..?? 화질?<br>
t21 -> ..??<br>
t2 -> 필기체<br>
t18 -> 화질<br>
t14 -> 아예 못읽어 드림(글씨체 문제인가?)<br>
t13 -> 글씨체..?<br>
t12 -> 글씨체<br>
t31 -> 어두움 + 글씨체<br>
t19 -> 어두움 + 안보임<br>
t8 -> 글씨 흐림 <br>
t6 -> 글씨체 + 흐림<br>
t40 -> 화질, 글씨체, 흐림<br>
t43 -> 화질, 흐림<br>
t39 -> 화질, 글씨 안보임<br>
t37 -> 글씨체<br>
t34 -> 글씨체 + 멀리서 찍음<br>
t35 -> 너무 어두움<br>
t30 -> 글씨체 + 흐림<br>
t27 -> 글씨체<br>

In [ ]:
# OCR4 TEST

for i in file_list:
  print(i[0])
  for j in i[1]:
    print(j)
    image = cv2.imread(j)

    MyOCR = OCR4(img=image) # i == image
    MyOCR.image_pytesseract()
  print("\n\n\n")